In [ ]:
import pandas as pd
from datasets import load_dataset
from collections import OrderedDict
from IPython.display import Audio
import librosa
import numpy as np

### Loading the Data

In [26]:
# List of language codes for the desired languages
language_codes = {
    "en": "English",
    "es": "Spanish",
    "ru": "Russian",
    "ja": "Japanese",
    "zh-CN": "Mandarin",
}

transcripts = []

for lang_code, lang_name in language_codes.items():
    try:
        dataset = load_dataset("mozilla-foundation/common_voice_11_0", lang_code, split="test", streaming=True)
        count = 0
        for x in dataset:
            if count >= 10:
                break
            count += 1
            transcripts.append({"language": lang_name, "sentence": x["sentence"], "audio": x["audio"]})
        print(f"Loaded transcripts for language: {lang_name}")
    except Exception as e:
        print(f"Error loading transcripts for language: {lang_name}. Error: {str(e)}")

df = pd.DataFrame(transcripts)
print(df)

Reading metadata...: 16354it [00:00, 47569.60it/s]


Loaded transcripts for language: English


Reading metadata...: 15520it [00:00, 56908.66it/s]


Loaded transcripts for language: Spanish


Reading metadata...: 9630it [00:00, 52724.93it/s]


Loaded transcripts for language: Russian


Reading metadata...: 4604it [00:00, 41074.53it/s]


Loaded transcripts for language: Japanese


Reading metadata...: 10581it [00:00, 51573.63it/s]


Loaded transcripts for language: Mandarin
    language                                           sentence  \
0    English  Joe Keaton disapproved of films, and Buster al...   
1    English                               She'll be all right.   
2    English                                                six   
3    English                         All's well that ends well.   
4    English  It is a busy market town that serves a large s...   
5    English  The team had Olympic Champion Carolina Marin i...   
6    English                                    Do you mean it?   
7    English  The new patch is less invasive than the old on...   
8    English  How is Mozilla going to handle ambiguities lik...   
9    English            I guess you must think I'm kinda batty.   
10   Spanish          Habita en aguas poco profundas y rocosas.   
11   Spanish  Opera principalmente vuelos de cabotaje y regi...   
12   Spanish                                               tres   
13   Spanish  Realiz

### Data Exploration

In [30]:
# play an audio to make sure its loaded properly

index_to_play = 0
audio_array = df.loc[index_to_play, "audio"]["array"]
sampling_rate = df.loc[index_to_play, "audio"]["sampling_rate"]

print(df.loc[index_to_play, 'sentence'])
display(Audio(np.asarray(audio_array), rate=sampling_rate))

Joe Keaton disapproved of films, and Buster also had reservations about the medium.
